In [1]:
from api import API
import json

In [2]:
api = API()

In [3]:
def get_rooms_points(buildings):
    """maps building ids to their points and rooms,
    ie {4:{'rooms':{5}}}"""
    result = {}
    for building_id, name in buildings.items():
        building_data = {
            'rooms': map_rooms(api.building_rooms(building_id)),
            'points': map_points(api.building_points(building_id))
        }
        result[building_id] = building_data
    return result

def map_points(points):
    """Take a pandas df returned from building_points
    and turn it into something easier to use on the frontend
    """
    results = {}
    for index, row in points.iterrows():
        results[row['id']] = row['name'] + '- ' + row['description']
    return results


def map_rooms(rooms):
    """Similar to the map_points function but for rooms"""
    results = {}
    for index, row in rooms.iterrows():
        results[row['id']] = row['name'].replace("_", " ")
    return results

In [6]:
api.building_points('4')

,description,id,name,pointsourceid,pointtypeid,roomid
0,LAB HOT H2O TEMP,511,BI.LAB.HW.TEMP,2,13,6
1,A1 CCL TEMP,512,BI1CCT,2,13,6
2,AH2 AIR VOLU,513,BI1CFM,2,47,6
3,STATIC SET,514,BI1DSC,2,51,6
4,A1 SA STATIC,515,BI1DSP,2,51,6
5,ZONE HUM SET,516,BI1HMC,2,82,6
6,A1 HTREC TMP,517,BI1HRT,2,13,6
7,HT REC RET T,518,BI1RRC,2,13,6
8,A1 HREC RTMP,519,BI1RRT,2,13,6
9,SUP AIR SET,520,BI1SAC,2,13,6


In [4]:
building_names = api.buildings()
rooms_points = get_rooms_points(building_names)

In [45]:
rooms_points

{1: {'points': {1: 'ACDIN.AH1.CCV- AH1 CLG VLV',
   2: 'ACDIN.AH1.DSP- AH1 SA PRESS',
   3: 'ACDIN.AH1.DSSP- DUCT STATIC STPT',
   4: 'ACDIN.AH1.ERR- SPLY TEMP ERROR',
   5: 'ACDIN.AH1.FBD- AHI FB DPRS',
   6: 'ACDIN.AH1.HCV- AH1 HTG VLV',
   7: 'ACDIN.AH1.LP1- MIX AIR LOOPOUT',
   8: 'ACDIN.AH1.LP2- RET FAN RESET',
   9: 'ACDIN.AH1.LPO- SUP TEMP LOOPOUT',
   10: 'ACDIN.AH1.MAD- AH1 MA DPRS',
   11: 'ACDIN.AH1.MAM- MIN DPR POS',
   12: 'ACDIN.AH1.MAS- MIX AIR SETPNT',
   13: 'ACDIN.AH1.MAT- AH1 MA TEMP',
   14: 'ACDIN.AH1.MAX.DMPR.COMD- MAX VAV DMPR',
   15: 'ACDIN.AH1.MOAD- AH1 MIN OAD',
   16: 'ACDIN.AH1.MOD- AH1 MODE',
   17: 'ACDIN.AH1.OACFM- AH1 OA CFM',
   18: 'ACDIN.AH1.OAT- VIR OAT',
   19: 'ACDIN.AH1.RADPT- RTRN DEW POINT',
   20: 'ACDIN.AH1.RAH- AH1 RA HUMID',
   21: 'ACDIN.AH1.RAT- AH1 RA TEMP',
   22: 'ACDIN.AH1.RDPS- CL DN SPLY SETPT',
   23: 'ACDIN.AH1.RFNVFD- AH1 RFN VFD',
   24: 'ACDIN.AH1.SAS- SUPPLY AIR SETPT',
   25: 'ACDIN.AH1.SAT- AH1 SA TEMP',
   26: 'ACDIN.AH1.SF

In [6]:
def rooms_points_json(rooms_points):
    return '<script type="text/javascript">var rooms_points =' + json.dumps(rooms_points) + ';</script>'

In [ ]:
poi

In [49]:
rooms_points

{2: {'points': {348: 'LIA1DA- DIS AIR TEMP',
   349: 'LIA1DC- MIX AIR DMPR',
   350: 'LIA1DS- DIS AIR STPT',
   351: 'LIA1MS- MIX AIR STPT',
   352: 'LIA1MT- MIX AIR TEMP',
   353: 'LIA1ST- SPACE TEMP',
   354: 'LIA1SV- HEATING VALVE',
   355: 'LIA2DA- DIS AIR TEMP',
   356: 'LIA2DC- MIX AIR DMPR',
   357: 'LIA2DS- DIS AIR STPT',
   358: 'LIA2MS- MIX AIR STPT',
   359: 'LIA2MT- MIX AIR TEMP',
   360: 'LIA2ST- SPACE TEMP',
   361: 'LIA2SV- STEAM VALVE',
   362: 'LIA3DA- DIS AIR TEMP',
   363: 'LIA3DC- MIX AIR DMPR',
   364: 'LIA3DS- DIS AIR STPT',
   365: 'LIA3MS- MIX AIR STPT',
   366: 'LIA3MT- MIX AIR TEMP',
   367: 'LIA3SV- HEATING VALVE',
   368: 'LIA4DA- DIS AIR TEMP',
   369: 'LIA4DC- MIX AIR DMPR',
   370: 'LIA4DS- DIS AIR STPT',
   371: 'LIA4MS- MIX AIR STPT',
   372: 'LIA4MT- MIX AIR TEMP',
   373: 'LIA4ST- SPACE TEMP',
   374: 'LIA4SV- STEAM VALVE',
   375: 'LIA5CV- COOLING CNTRL',
   376: 'LIA5DA- DIS AIR TEMP',
   377: 'LIA5DC- MIX AIR DMPR',
   378: 'LIA5HC- HEATING CNTRL',

In [50]:
json.dumps(rooms_points)

TypeError: keys must be a string

In [7]:
json_res = rooms_points_json(rooms_points)

TypeError: keys must be a string